In [0]:
#!import "../DataModel/DataStructure"

In [0]:
public record KeyedActivity : KeyedRecord
{
    public string Username {get; init;}

    public DateTime StartDateTime {get; init;}

    public DateTime EndDateTime {get; init;}

    public ActivityLogStatus Status {get; init;}

    public string Category {get; init;}

    public string Format {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] ErrorMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] WarningMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] InfoMessages {get; init;}

    public string VariableType {get; init;}

    public string FileId {get; init;}

}

In [0]:
public record KeyedFile : KeyedRecord
{
    public string Name {get; init;}

    public DateTime CreationTime {get; init;}

    public byte[] SerializedFile {get; init;}

    public string Directory {get; init;}

    public long FileLength {get; init;}
    
    public string ContentType {get; init;}

    public string Partition {get; init;}

    public string Source {get; init;}
}

In [0]:
using System.IO;
using System.Text;
using Systemorph.Vertex.DataSetReader.Csv;
using Systemorph.Vertex.Session;
using Systemorph.Vertex.Import.Builders;
using Systemorph.Vertex.Import.Mappings;
using Systemorph.Vertex.DataSetReader;

public record StreamWrapper(Stream Stream, bool WillBeReused);

public static string ProcessNotification(this object obj) => obj is ActivityMessageNotification amn ? amn.Message : ""; 

public static async Task<ActivityLog> ExecuteWithStoreActivityAsync(this ImportOptionsBuilder builder, 
                                                                                ISessionVariable session, 
                                                                                IDataSource dataSource)
{
    var log = await builder.ExecuteAsync();
    var options = builder.GetImportOptions();
    string formatFromFile;
    byte[] content;
    StreamWrapper stream = options switch
    {
        FileImportOptions fio => new StreamWrapper(await fio.Storage.ReadAsync(fio.FileName, session.CancellationToken), true),
        StreamImportOptions streamImportOptions => new StreamWrapper(streamImportOptions.Stream, false),
        StringImportOptions stringImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(stringImportOptions.Content)), true),
        DataSetImportOptions dataSetImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(DataSetCsvSerializer.Serialize(dataSetImportOptions.DataSet))), true),
        _ => null
    };
    using (MemoryStream ms = new MemoryStream())
    {
        await stream.Stream.CopyToAsync(ms);
        content = ms.ToArray();
        ms.Position = 0;
        DataSetImportVariable importVariable = new DataSetImportVariable(session, sess => sess.CancellationToken);
        var dsRes = await importVariable.ReadFromStream(ms).ExecuteAsync();
        formatFromFile = dsRes.Format;
        if (stream.WillBeReused)
            stream.Stream.Position = 0;
        else
        {
            stream.Stream.Close();
            await stream.Stream.DisposeAsync();
        }
    }
    string fileName = options is FileImportOptions fileImportOptions ? fileImportOptions.FileName : null; 
    var format = formatFromFile ?? options.Format;
    var keyedFile = new KeyedFile() with { Id = Guid.NewGuid(),
                                        SerializedFile = content, 
                                        Name = Path.GetFileName(fileName), 
                                        CreationTime = DateTime.UtcNow,
                                        Directory = Path.GetDirectoryName(fileName), 
                                        ContentType = Path.GetExtension(fileName), 
                                        FileLength = content.Length,
                                        Partition = null, // Andrey Katz: Options.TargetDataSource.Partion.GetCurrent(?? What do we put here, different classes might posess various partitions, e.g. Yield Curve has none ??)
                                        Source = options.Storage.GetType().Name};
    var activity = new KeyedActivity() with { Id = Guid.NewGuid(),
                                            Username = session.User.Name, 
                                            StartDateTime = log.StartDateTime, 
                                            EndDateTime = log.FinishDateTime, 
                                            Status = log.Status, 
                                            ErrorMessages = log.Errors.Select(x => x.ProcessNotification()).ToArray(), 
                                            WarningMessages = log.Warnings.Select(x => x.ProcessNotification()).ToArray(), 
                                            InfoMessages = log.Infos.Select(x => x.ProcessNotification()).ToArray(), 
                                            Format = format,
                                            Category = "Import",
                                            FileId = keyedFile.Id.ToString()};
    await dataSource.UpdateAsync<KeyedFile>(keyedFile.RepeatOnce());
    await dataSource.UpdateAsync<KeyedActivity>(activity.RepeatOnce());
    await dataSource.CommitAsync(); 
    return log;
}

Thoughts about API. First: what do we save if the import is not from the File, but rather stream, string, or data set? Suggestions:
1. Need one more abstraction level: KeyedImport
2. KeyedImport should include: Guid Id, Type (string, stream, data set or file), creation time and serialized content and content length. The latter has nothing to do with the file per se, it is simply a byte length of the import. These are generic types.
3. KeyedFile should directly inherit from the Keyed Import, and this can include on top of that Directory, content type, name.
4. FileId in the KeyedActivity should probably be ImportId, as the import is not necesserily from a file. 
5. Name might be either be a file-only attribute, or be assigned autimatically if not given. In this case it might be an identity property.
   
Question: how do we accomodate this in our tables? We can have separate tables for strings, data sets and streams. If in U+ some of them will stay empty, this does not mean that they will not be populated in other projects. In fact, the type of import can be moved to the KeyedActivity.

On the API side. Certain actions must be moved to the KeyedImport and maybe even made private (in fact, they should not exposed to the outside user):
1. Determination of content and length. 
2. Creation date and time
3. Type of input. 
   
Prposed API: Initialize the import via options -> determine the content, time, and type, which are called automatically from constructor. 